In [ ]:
"""Find employees whose salaries are higher than their manager"""

In [ ]:
"""SQL table creation and data insertion"""
drop table if exists  `dept_emp` ; 
drop table if exists  `dept_manager` ; 
drop table if exists  `salaries` ; 

CREATE TABLE `dept_emp` (
`emp_no` int(11) NOT NULL,
`dept_no` char(4) NOT NULL,
`from_date` date NOT NULL,
`to_date` date NOT NULL,
PRIMARY KEY (`emp_no`,`dept_no`));

CREATE TABLE `dept_manager` (
`dept_no` char(4) NOT NULL,
`emp_no` int(11) NOT NULL,
`from_date` date NOT NULL,
`to_date` date NOT NULL,
PRIMARY KEY (`emp_no`,`dept_no`));

CREATE TABLE `salaries` (
`emp_no` int(11) NOT NULL,
`salary` int(11) NOT NULL,
`from_date` date NOT NULL,
`to_date` date NOT NULL,
PRIMARY KEY (`emp_no`,`from_date`));

INSERT INTO dept_emp VALUES(10001,'d001','1986-06-26','9999-01-01');
INSERT INTO dept_emp VALUES(10002,'d001','1996-08-03','9999-01-01');

INSERT INTO dept_manager VALUES('d001',10002,'1996-08-03','9999-01-01');

INSERT INTO salaries VALUES(10001,88958,'2002-06-22','9999-01-01');
INSERT INTO salaries VALUES(10002,72527,'1996-08-03','9999-01-01');

In [ ]:
"""SQL solution"""
SELECT a.emp_no, a.manager_no, a.emp_salary, a.manager_salary
FROM
(SELECT s.emp_no, s.salary AS emp_salary, de.dept_no, dm.emp_no AS manager_no, ms.salary AS manager_salary
FROM salaries AS s
INNER JOIN dept_emp AS de
ON s.emp_no = de.emp_no
INNER JOIN dept_manager AS dm
ON de.dept_no = dm.dept_no
INNER JOIN
(SELECT emp_no, salary
FROM salaries
WHERE emp_no IN (SELECT emp_no FROM dept_manager)) AS ms
ON dm.emp_no = ms.emp_no) AS a
WHERE a.emp_salary > a.manager_salary

In [17]:
"""pandas dataframe creation"""
import numpy as np
import pandas as pd

# this task requires only 3 tables, table employees is not necessary
t_dept_emp = np.array([[10001,'d001','1986-06-26','9999-01-01'],
                       [10002,'d001','1996-08-03','9999-01-01']])
t_sal = np.array([[10001,88958,'1986-06-26','9999-01-01'],
                  [10002,72527,'1996-08-03','9999-01-01']])

dept_emp = pd.DataFrame(data=t_dept_emp, columns=['emp_no', 'dept_no', 'from_date', 'to_date'])
salaries = pd.DataFrame(data=t_sal, columns=['emp_no', 'salary', 'from_date', 'to_date'])
dept_manager = pd.DataFrame(data=[['d001',10002,'1996-08-03','9999-01-01']], columns=['dept_no', 'emp_no', 'from_date', 'to_date'])
dept_emp['emp_no'] = dept_emp['emp_no'].astype('int')
dept_emp.head()

,emp_no,dept_no,from_date,to_date
0,10001,d001,1986-06-26,9999-01-01
1,10002,d001,1996-08-03,9999-01-01


In [22]:
salaries['emp_no'] = salaries['emp_no'].astype('int')
salaries.head()

,emp_no,salary,from_date,to_date,emo_no
0,10001,88958,1986-06-26,9999-01-01,10001
1,10002,72527,1996-08-03,9999-01-01,10002


In [3]:
dept_manager.head()

,dept_no,emp_no,from_date,to_date
0,d001,10002,1996-08-03,9999-01-01


In [24]:
# find employees with their salaries
emp_sal = pd.merge(dept_emp[['emp_no', 'dept_no']], salaries[['emp_no', 'salary']], on='emp_no', how='inner')
emp_sal

,emp_no,dept_no,salary
0,10001,d001,88958
1,10002,d001,72527


In [26]:
# find managers with their salaries
man_sal = pd.merge(dept_manager[['emp_no', 'dept_no']], salaries[['emp_no', 'salary']], on='emp_no', how='inner')
man_sal

,emp_no,dept_no,salary
0,10002,d001,72527


In [35]:
# connect employees and their managers and compare their salaries
df = pd.merge(emp_sal, man_sal, on='dept_no', how='inner', suffixes=['', '_manager'])
result = df[df['salary'] > df['salary_manager']][['emp_no', 'emp_no_manager', 'salary', 'salary_manager']]
result.rename(columns={'emp_no_manager': 'manager_no', 'salary': 'emp_salary', 'salary_manager': 'manager_salary'})

,emp_no,manager_no,emp_salary,manager_salary
0,10001,10002,88958,72527
